<a href="https://colab.research.google.com/github/PadmarajBhat/Real-Time-Analytics-on-Hadoop/blob/master/Elephas_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U pyspark

     |████████████████████████████████| 215.6MB 113kB/s 
     |████████████████████████████████| 204kB 37.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/8d/20/f0/b30e2024226dc112e256930dd2cd4f06d00ab053c86278dcf3
Successfully built pyspark


In [3]:
!pip install -U pyarrow

Requirement already up-to-date: pyarrow in /usr/local/lib/python3.6/dist-packages (0.13.0)


In [4]:
!sudo apt install openjdk-8-jdk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libxxf86dga1 openjdk-8-jre x11-utils
Suggested packages:
  gvfs openjdk-8-demo openjdk-8-source visualvm icedtea-8-plugin mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libxxf86dga1 openjdk-8-jdk openjdk-8-jre x11-utils
0 upgraded, 13 newly installed, 0 to remove and 6 not upgraded.
Need to get 6,847 kB of archives.
After this operation, 19.8 MB of additional disk space will be used.
Get:

In [5]:
!sudo update-alternatives --config java

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
* 0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
  2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 2
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


In [6]:
!pip install -U elephas

     |████████████████████████████████| 337kB 7.8MB/s 
ERROR: ipython 5.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.4, but you'll have prompt-toolkit 2.0.9 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.16
    Uninstalling prompt-toolkit-1.0.16:
      Successfully uninstalled prompt-toolkit-1.0.16


In [7]:
from __future__ import absolute_import
from __future__ import print_function

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras import optimizers

from elephas.ml_model import ElephasEstimator
from elephas.ml.adapter import to_data_frame

from pyspark import SparkContext, SparkConf
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml import Pipeline


# Define basic parameters
batch_size = 16
nb_classes = 10
epochs = 1





WARNING


In [39]:
%%timeit

# Load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_test.shape, y_test.shape, x_train.shape, y_train.shape, type(x_train), type(y_train))

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)


train_size = 5000
test_size = 500
x_train = x_train[:train_size,:]
y_train = y_train[:train_size]
x_test = x_test[:test_size,:]
y_test = y_test[:test_size]



x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


print(x_test.shape, y_test.shape, x_train.shape, y_train.shape, type(x_train), type(y_train))

# Convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

print(type(y_train), type(y_test))
print(y_train[:20])

model = Sequential()
model.add(Dense(128, input_dim=784))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))

# Create Spark context
try:
    sc.stop()
    spark.stop()
except:
    pass

conf = SparkConf().setAppName('Mnist_Spark_MLP').setMaster('local[3]').set("spark.executor.heartbeatInterval","3600s").\
set("spark.network.timeout","3601s").set("spark.executor.memory","1GB").set("spark.executor.pyspark.memory","2GB")
sc = SparkContext(conf=conf)

'''conf = ps.SparkConf().setMaster("yarn-client").setAppName("sparK-mer")
conf.set("spark.executor.heartbeatInterval","3600s")
sc = ps.SparkContext('local[4]', '', conf=conf) # uses 4 cores on your local machine'''

# Build RDD from numpy features and labels
df = to_data_frame(sc, x_train, y_train, categorical=True)
df = df.repartition(5)

test_df = to_data_frame(sc, x_test, y_test, categorical=True)

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
sgd_conf = optimizers.serialize(sgd)

# Initialize Spark ML Estimator
estimator = ElephasEstimator()
estimator.set_keras_model_config(model.to_yaml())
estimator.set_optimizer_config(sgd_conf)
estimator.set_mode("synchronous")
estimator.set_loss("categorical_crossentropy")
estimator.set_metrics(['acc'])
estimator.set_epochs(epochs)
estimator.set_batch_size(batch_size)
estimator.set_validation_split(0.1)
estimator.set_categorical_labels(True)
estimator.set_nb_classes(nb_classes)

# Fitting a model returns a Transformer
pipeline = Pipeline(stages=[estimator])
fitted_pipeline = pipeline.fit(df)


#model.compile(loss="categorical_crossentropy", optimizer=sgd_conf, metrics=["accuracy"])
#model.fit(x_train,y_train)

# Evaluate Spark model by evaluating the underlying model
prediction = fitted_pipeline.transform(test_df)
#prediction = model.predict(x_test)
pnl = prediction.select("label", "prediction")
pnl.show(100)

prediction_and_label = pnl.rdd.map(lambda row: (row.label, row.prediction))
metrics = MulticlassMetrics(prediction_and_label)
print(metrics.precision())
print(metrics.recall())

(10000, 28, 28) (10000,) (60000, 28, 28) (60000,) <class 'numpy.ndarray'> <class 'numpy.ndarray'>
5000 train samples
500 test samples
(500, 784) (500,) (5000, 784) (5000,) <class 'numpy.ndarray'> <class 'numpy.ndarray'>
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


ValueError: ignored

##### Observations:
* Input mnist data got split in training and testing **numpy** array
* a utility function in numpy convers the lables to one hot encoding. Note here it is converting it to 10 labels. However, udacity training notes that the one hot encoding should result in N-1 columns for N lables.

* Now , to_data_frame, converts the numpy array to spark data frames. So what is the difference between it and createDataFrame function ? Well, we cannot give the numpy directly to createDataFrame, as indicated below.

In [24]:
df.describe()

DataFrame[summary: string, label: string]

In [25]:
df.printSchema()

root
 |-- features: vector (nullable = true)
 |-- label: double (nullable = true)



In [30]:
df.show(truncate=True)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.0,0.0,0.0,0.0,...|  5.0|
|[0.0,0.0,0.0,0.0,...|  0.0|
|[0.0,0.0,0.0,0.0,...|  4.0|
|[0.0,0.0,0.0,0.0,...|  1.0|
|[0.0,0.0,0.0,0.0,...|  9.0|
|[0.0,0.0,0.0,0.0,...|  2.0|
|[0.0,0.0,0.0,0.0,...|  1.0|
|[0.0,0.0,0.0,0.0,...|  3.0|
|[0.0,0.0,0.0,0.0,...|  1.0|
|[0.0,0.0,0.0,0.0,...|  4.0|
|[0.0,0.0,0.0,0.0,...|  3.0|
|[0.0,0.0,0.0,0.0,...|  5.0|
|[0.0,0.0,0.0,0.0,...|  3.0|
|[0.0,0.0,0.0,0.0,...|  6.0|
|[0.0,0.0,0.0,0.0,...|  1.0|
|[0.0,0.0,0.0,0.0,...|  7.0|
|[0.0,0.0,0.0,0.0,...|  2.0|
|[0.0,0.0,0.0,0.0,...|  8.0|
|[0.0,0.0,0.0,0.0,...|  6.0|
|[0.0,0.0,0.0,0.0,...|  9.0|
+--------------------+-----+
only showing top 20 rows



In [31]:
print(x_test.shape, y_test.shape, x_train.shape, y_train.shape, type(x_train), type(y_train))

(500, 784) (500, 10) (500, 784) (500, 10) <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("abcd").getOrCreate()

In [32]:
df_2 = spark.createDataFrame(x_train, y_train,['features','labesl'])
df_2.printSchema()

TypeError: ignored

In [22]:
x_train.reshape()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)